# Modelo Preditivo com Regressão Logística - Dataset Titanic

## 1. Carregamento dos Dados

In [5]:
import pandas as pd

# Carregar o dataset de treino (contém 'Survived')
train_df = pd.read_csv('train.csv')

# Carregar o dataset de teste (não contém 'Survived')
test_df = pd.read_csv('test.csv')

# Carregar o arquivo de submissão (rótulos verdadeiros para o teste)
submission_true = pd.read_csv('gender_submission.csv')

print('Treino:', train_df.shape)
print('Teste:', test_df.shape)
print('Submission:', submission_true.shape)

Treino: (891, 12)
Teste: (418, 11)
Submission: (418, 2)


## 2. Pré-processamento dos Dados

Nesta etapa, vamos tratar valores ausentes, remover colunas irrelevantes e converter variáveis categóricas para formato numérico. 

### 2.1 Pré-processamento do dataset de treino

In [6]:
# Preencher valores ausentes na coluna 'Age' com a mediana
train_df['Age'].fillna(train_df['Age'].median(), inplace=True)

# Preencher valores ausentes na coluna 'Embarked' com a moda
train_df['Embarked'].fillna(train_df['Embarked'].mode()[0], inplace=True)

# Remover colunas que não serão utilizadas: 'Cabin', 'Ticket', 'Name'
train_df.drop(['Cabin', 'Ticket', 'Name'], axis=1, inplace=True)

# Converter a coluna 'Sex' para numérico: male -> 1, female -> 0
train_df['Sex'] = train_df['Sex'].map({'male': 1, 'female': 0})

# Criar a feature 'FamilySize' (número de familiares a bordo)
train_df['FamilySize'] = train_df['SibSp'] + train_df['Parch']

# Aplicar one-hot encoding para a coluna 'Embarked'
train_df = pd.get_dummies(train_df, columns=['Embarked'], drop_first=True)

train_df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,FamilySize,Embarked_Q,Embarked_S
0,1,0,3,1,22.0,1,0,7.2500,1,0,1
1,2,1,1,0,38.0,1,0,71.2833,1,0,0
2,3,1,3,0,26.0,0,0,7.9250,0,0,1
3,4,1,1,0,35.0,1,0,53.1000,1,0,1
4,5,0,3,1,35.0,0,0,8.0500,0,0,1


### 2.2 Pré-processamento do dataset de teste

Realizamos os mesmos passos de pré-processamento no dataset de teste.

In [7]:
# Preencher valores ausentes na coluna 'Age' com a mediana
test_df['Age'].fillna(test_df['Age'].median(), inplace=True)

# Preencher valores ausentes na coluna 'Fare' (caso existam) com a mediana
test_df['Fare'].fillna(test_df['Fare'].median(), inplace=True)

# Preencher valores ausentes na coluna 'Embarked' com a moda
test_df['Embarked'].fillna(test_df['Embarked'].mode()[0], inplace=True)

# Remover colunas: 'Cabin', 'Ticket', 'Name'
test_df.drop(['Cabin', 'Ticket', 'Name'], axis=1, inplace=True)

# Converter a coluna 'Sex' para numérico
test_df['Sex'] = test_df['Sex'].map({'male': 1, 'female': 0})

# Criar a feature 'FamilySize'
test_df['FamilySize'] = test_df['SibSp'] + test_df['Parch']

# Aplicar one-hot encoding para a coluna 'Embarked'
test_df = pd.get_dummies(test_df, columns=['Embarked'], drop_first=True)

test_df.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,FamilySize,Embarked_Q,Embarked_S
0,892,3,1,34.5,0,0,7.8292,0,1,0
1,893,3,0,47.0,1,0,7.0000,1,0,1
2,894,2,1,62.0,0,0,9.6875,0,1,0
3,895,3,1,27.0,0,0,8.6625,0,0,1
4,896,3,0,22.0,1,1,12.2875,2,0,1


## 3. Treinamento do Modelo

Utilizaremos o dataset de treino para ajustar um modelo de regressão logística.

Como a coluna `PassengerId` não é uma feature útil, vamos removê-la. O mesmo vale para a variável target que está em `Survived`.

In [8]:
# Preparar os dados de treino removendo 'PassengerId' e separando a variável target
X_train = train_df.drop(['PassengerId', 'Survived'], axis=1)
y_train = train_df['Survived']

from sklearn.linear_model import LogisticRegression

# Instanciar e treinar o modelo
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

print("Modelo treinado com sucesso!")

Modelo treinado com sucesso!


## 4. Geração de Predições e Criação do Arquivo de Submissão

Vamos usar o modelo treinado para gerar predições no dataset de teste e, em seguida, criar um arquivo de submissão. 

Obs.: Removemos a coluna `PassengerId` do conjunto de teste para predição, mas ela será incluída no arquivo final.

In [9]:
# Preparar os dados de teste removendo a coluna 'PassengerId' (somente para predição)
X_test = test_df.drop('PassengerId', axis=1)

# Gerar as predições para o conjunto de teste
predictions = model.predict(X_test)

# Criar um DataFrame para a submissão
submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Survived': predictions
})

# Salvar o arquivo de submissão
submission.to_csv('submission.csv', index=False)
print("Arquivo submission.csv gerado com sucesso!")

Arquivo submission.csv gerado com sucesso!


## 5. Avaliação do Modelo (Auto-Avaliação)

Para avaliar a acurácia das predições, vamos comparar o arquivo `submission.csv` (nossas predições) com o arquivo `gender_submission.csv`, que contém os rótulos verdadeiros para os dados de teste. 

Utilizaremos métricas como acurácia, matriz de confusão e relatório de classificação.

In [10]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Carregar o arquivo de submissão verdadeiro (gender_submission.csv)
true_submission = pd.read_csv('gender_submission.csv')

# Carregar o arquivo gerado pelo modelo
pred_submission = pd.read_csv('submission.csv')

# Fazer o merge dos dois DataFrames com base em PassengerId
merged = pd.merge(true_submission, pred_submission, on='PassengerId', how='inner')

# Calcular a acurácia
accuracy = accuracy_score(merged['Survived_x'], merged['Survived_y'])
print("Acurácia das predições:", accuracy)

# Exibir a matriz de confusão
cm = confusion_matrix(merged['Survived_x'], merged['Survived_y'])
print("Matriz de Confusão:\n", cm)

# Exibir relatório de classificação
report = classification_report(merged['Survived_x'], merged['Survived_y'])
print("Relatório de Classificação:\n", report)

Acurácia das predições: 0.9330143540669856
Matriz de Confusão:
 [[248  18]
 [ 10 142]]
Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.96      0.93      0.95       266
           1       0.89      0.93      0.91       152

    accuracy                           0.93       418
   macro avg       0.92      0.93      0.93       418
weighted avg       0.93      0.93      0.93       418



## 6. Conclusão

Neste notebook, vimos como:

- Realizar o pré-processamento dos dados (tratamento de valores ausentes, conversão de variáveis categóricas e criação de novas features).
- Treinar um modelo de regressão logística utilizando o dataset de treino.
- Gerar predições para o dataset de teste e criar um arquivo de submissão.
- Avaliar as predições comparando com os rótulos verdadeiros do arquivo `gender_submission.csv`.